In [1]:
# SEL 1: Setup & Install Dependencies

!pip uninstall -y transformers evaluate datasets huggingface_hub gradio
!pip install openai
!pip install keybert
!pip install --no-cache-dir \
    transformers==4.44.2 \
    evaluate==0.4.2 \
    datasets==3.0.2 \
    ipywidgets==8.1.1 \
    sentence_transformers \
    gradio \
    "huggingface_hub>=0.33.5" \
    typing_extensions

print("--- Instalasi Selesai ---")

Found existing installation: transformers 4.44.2
Uninstalling transformers-4.44.2:
  Successfully uninstalled transformers-4.44.2
Found existing installation: evaluate 0.4.2
Uninstalling evaluate-0.4.2:
  Successfully uninstalled evaluate-0.4.2
Found existing installation: datasets 3.0.2
Uninstalling datasets-3.0.2:
  Successfully uninstalled datasets-3.0.2
Found existing installation: huggingface-hub 0.36.0
Uninstalling huggingface-hub-0.36.0:
  Successfully uninstalled huggingface-hub-0.36.0
Found existing installation: gradio 6.0.1
Uninstalling gradio-6.0.1:
  Successfully uninstalled gradio-6.0.1
  Using cached transformers-4.57.3-py3-none-any.whl.metadata (43 kB)
  Using cached huggingface_hub-1.1.5-py3-none-any.whl.metadata (13 kB)
  Using cached huggingface_hub-0.36.0-py3-none-any.whl.metadata (14 kB)
  Using cached tokenizers-0.22.1-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.8 kB)
Using cached transformers-4.57.3-py3-none-any.whl (12.0 MB)
Using cached

In [2]:
# SEL 2: Import Basic Libraries & List Files

import numpy as np
import pandas as pd
import os

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

print("\n✅ Basic libraries imported successfully!")

/kaggle/input/asap-aes/valid_sample_submission_1_column.csv
/kaggle/input/asap-aes/Training_Materials.zip
/kaggle/input/asap-aes/training_set_rel3.xls
/kaggle/input/asap-aes/valid_sample_submission_1_column_no_header.csv
/kaggle/input/asap-aes/Essay_Set_Descriptions.zip
/kaggle/input/asap-aes/training_set_rel3.xlsx
/kaggle/input/asap-aes/valid_set.xls
/kaggle/input/asap-aes/training_set_rel3.tsv
/kaggle/input/asap-aes/valid_sample_submission_5_column.csv
/kaggle/input/asap-aes/valid_set.xlsx
/kaggle/input/asap-aes/valid_set.tsv
/kaggle/input/asap-aes/test_set.tsv
/kaggle/input/asap-aes/valid_sample_submission_2_column.csv

✅ Basic libraries imported successfully!


In [3]:
# SEL 3: Configuration


import torch

configs = {
    "device": torch.device("cuda" if torch.cuda.is_available() else 'cpu'),
    "data_path": "/kaggle/input/asap-aes/training_set_rel3.tsv",
    "eval_path": "/kaggle/input/asap-aes/test_set.tsv",
    "save_path": "models",
    "tokenizer_name": "bert-base-uncased",
    "model_name": "bert-base-uncased",
    "MAX_LENGTH": 2048,
    "BATCH_SIZE": 4,
    "EPOCHS": 4,
    "patience": 2,
    "learning_rate": 2e-4,
    "weight_decay": 0.01
}

print(f"🖥️ Device: {configs['device']}")
print("✅ Configuration loaded!")


🖥️ Device: cuda
✅ Configuration loaded!


In [4]:
# SEL 4: Load & Split Data

from sklearn.model_selection import train_test_split

print("📂 Loading dataset...")
training_data1 = pd.read_csv(
    "/kaggle/input/asap-aes/training_set_rel3.tsv",
    sep='\t', 
    encoding='ISO-8859-1',
    usecols=['essay_id', 'essay_set', 'essay', 'domain1_score']
).dropna(axis=1)

print(f"✅ Dataset loaded: {training_data1.shape}")
print(f"Columns: {training_data1.columns.tolist()}")

training_data, test_data = train_test_split(
    training_data1, 
    test_size=0.2, 
    random_state=42
)

print(f"📊 Train set: {training_data.shape}")
print(f"📊 Test set: {test_data.shape}")
print("\nSample data:")
print(training_data.head(3))

📂 Loading dataset...
✅ Dataset loaded: (12976, 4)
Columns: ['essay_id', 'essay_set', 'essay', 'domain1_score']
📊 Train set: (10380, 4)
📊 Test set: (2596, 4)

Sample data:
      essay_id  essay_set                                              essay  \
2372      3567          2  There are many types of reading materials for ...   
3038      4233          2  All terms of offense are based on an opinion. ...   
4122      6518          3  The features of the setting affected the cycal...   

      domain1_score  
2372              4  
3038              3  
4122              1  


In [5]:
#  SEL 5: Import All ML/DL Libraries

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from xgboost import XGBRegressor
from sklearn.metrics import cohen_kappa_score, mean_squared_error
from sklearn.model_selection import train_test_split


import time
from tqdm import tqdm

print("✅ All ML/DL libraries imported!")

✅ All ML/DL libraries imported!


In [6]:
#  SEL 6: Load SBERT Model

from sentence_transformers import SentenceTransformer
from keybert import KeyBERT


print("🔄 Loading SBERT model...")
sbert_model = SentenceTransformer("paraphrase-MiniLM-L6-v2")
print("✅ SBERT model loaded: paraphrase-MiniLM-L6-v2")
kw_model = KeyBERT(model=sbert_model)


2025-11-27 00:03:15.837428: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764201796.030630     147 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764201796.086155     147 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

✅ SBERT model loaded: paraphrase-MiniLM-L6-v2


In [7]:
#  SEL 7: Keyword Summarization Preprocessing 

from keybert import KeyBERT

print("🔄 Initializing KeyBERT using SBERT embeddings...")
kw_model = KeyBERT(model=sbert_model)
print("✅ KeyBERT Ready!")


def keyword_summarize(text, top_k=5):
    """
    Ambil keyword dari teks — kompatibel dengan semua versi KeyBERT.
    """
    keywords = kw_model.extract_keywords(
        text,
        keyphrase_ngram_range=(1, 2),
        stop_words='english'
    )
    
    keywords = keywords[:top_k]

    key_list = [kw for kw, score in keywords]

    return " ".join(key_list) if key_list else text


🔄 Initializing KeyBERT using SBERT embeddings...
✅ KeyBERT Ready!


In [8]:
#  SEL 8: COMPLETE TRAINING & COMPARISON

print("="*80)
print("🚀 STARTING COMPLETE MODEL TRAINING & COMPARISON")
print("="*80)

#  HELPER FUNCTIONS

def qwk_score(y_true, y_pred, weights='quadratic'):
    """Calculate Quadratic Weighted Kappa (QWK)."""
    y_pred_int = np.rint(y_pred).astype(int)
    
    if len(y_true) > 0:
        min_score = min(y_true)
        max_score = max(y_true)
        y_pred_int = np.clip(y_pred_int, min_score, max_score)
        return cohen_kappa_score(y_true, y_pred_int, weights=weights)
    return 0.0

def get_sentence_embeddings(essay_text, show_progress_bar=False):
    """Get sentence embeddings from essay."""
    sentences = essay_text.split('. ')
    sentence_embeddings = sbert_model.encode(
        sentences, 
        convert_to_tensor=True, 
        show_progress_bar=show_progress_bar
    )
    return sentence_embeddings

def calculate_all_metrics(y_true, y_pred):
    """Calculate all evaluation metrics."""
    from sklearn.metrics import r2_score, mean_absolute_error
    
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    qwk = qwk_score(y_true, y_pred)
    
    return {
        'MSE': round(mse, 4),
        'R2': round(r2, 4),
        'MAE': round(mae, 4),
        'RMSE': round(rmse, 4),
        'QWK': round(qwk, 4)
    }

🚀 STARTING COMPLETE MODEL TRAINING & COMPARISON


In [9]:
# sel 9 PREPARE DATA (WITH KEYWORD SUMMARIZATION)

from keybert import KeyBERT

print("\n[STEP 1] Preparing data...")

kw_model = KeyBERT(model=sbert_model)   

def keyword_summarize(text, top_k=5):
    """
    Mengambil keyword penting dari teks dan mengembalikan
    ringkasan singkat untuk membantu model memahami konteks.
    """
    keywords = kw_model.extract_keywords(
        text,
        keyphrase_ngram_range=(1, 2),
        stop_words='english'
        
    )
    
    keywords_filtered = keywords[:top_k] 

    key_list = [kw for kw, score in keywords_filtered]
    
    if len(key_list) == 0:
        return text
    
    return " ".join(key_list)


print("Applying keyword summarization...")
essays_raw = training_data['essay'].tolist()
scores = training_data['domain1_score'].tolist()

essays = []
for t in tqdm(essays_raw, desc="Summarizing essays"):
    essays.append(keyword_summarize(t, top_k=5))


# Create embeddings using SBERT

print("Creating sentence embeddings...")
essay_embeddings = []
for essay in tqdm(essays, desc="Processing embeddings"):
    emb = get_sentence_embeddings(essay, show_progress_bar=False)
    essay_embeddings.append(emb)

padded_embeddings = nn.utils.rnn.pad_sequence(essay_embeddings, batch_first=True)
essay_scores = torch.tensor(scores, dtype=torch.float32).view(-1, 1)

print(f"✅ Embeddings shape: {padded_embeddings.shape}")


X_train, X_val, y_train, y_val = train_test_split(
    padded_embeddings, 
    essay_scores, 
    test_size=0.2, 
    random_state=42
)

print(f"Train: {X_train.shape}, Val: {X_val.shape}")

# Prepare for ML models (flatten)
X_train_np = X_train.cpu().numpy()
y_train_np = y_train.cpu().numpy().flatten()

# Mean pooling
mask = X_train_np != 0
count = np.sum(mask, axis=1)
count[count == 0] = 1
X_train_sum = np.sum(X_train_np, axis=1)
X_train_flat = X_train_sum / count

print(f"Flat features shape: {X_train_flat.shape}")



[STEP 1] Preparing data...
Applying keyword summarization...


Summarizing essays: 100%|██████████| 10380/10380 [07:41<00:00, 22.48it/s]


Creating sentence embeddings...


Processing embeddings: 100%|██████████| 10380/10380 [00:58<00:00, 177.71it/s]


✅ Embeddings shape: torch.Size([10380, 1, 384])
Train: torch.Size([8304, 1, 384]), Val: torch.Size([2076, 1, 384])
Flat features shape: (8304, 384)


In [10]:
# sel 10 DEFINE DEEP LEARNING ARCHITECTURES

EMBEDDING_DIM = X_train.shape[2]
HIDDEN_DIM = 128
OUTPUT_DIM = 1
NUM_EPOCHS = 10
DL_BATCH_SIZE = 64

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"\n[STEP 2] Using device: {device}")

# LSTM Model
class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(hidden_dim * 2, output_dim)
    
    def forward(self, x):
        _, (hidden, _) = self.lstm(x)
        hidden = torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1)
        return self.fc(hidden)

# CNN Model
class CNNModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv1d(input_dim, hidden_dim, kernel_size=3, padding=1)
        self.relu = nn.ReLU()
        self.pool = nn.AdaptiveMaxPool1d(1)
        self.fc = nn.Linear(hidden_dim, output_dim)
    
    def forward(self, x):
        x = x.permute(0, 2, 1)
        x = self.conv1(x)
        x = self.relu(x)
        x = self.pool(x).squeeze(2)
        return self.fc(x)

# LSTM-Attention Model
class LSTMAttentionModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(LSTMAttentionModel, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, batch_first=True, bidirectional=True)
        self.attention_weights = nn.Linear(hidden_dim * 2, 1)
        self.fc = nn.Linear(hidden_dim * 2, output_dim)

    def forward(self, x):
        outputs, _ = self.lstm(x)
        attn_scores = self.attention_weights(outputs)
        attn_scores = torch.tanh(attn_scores)
        attn_weights = torch.softmax(attn_scores, dim=1)
        context = torch.sum(attn_weights * outputs, dim=1)
        return self.fc(context)

print("✅ Deep Learning architectures defined!")


[STEP 2] Using device: cuda
✅ Deep Learning architectures defined!


In [11]:
# ssel 11 TRAINING FUNCTION FOR DL MODELS

def train_dl_model(model, X_train_dl, y_train_dl, num_epochs=NUM_EPOCHS):
    model.to(device)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=1e-3)
    
    dataset = torch.utils.data.TensorDataset(
        X_train_dl.to(device), 
        y_train_dl.to(device)
    )
    dataloader = torch.utils.data.DataLoader(
        dataset, 
        batch_size=DL_BATCH_SIZE, 
        shuffle=True
    )
    
    start_time = time.time()
    
    for epoch in tqdm(range(num_epochs), desc=f"Training {model.__class__.__name__}"):
        model.train()
        for X_batch, y_batch in dataloader:
            optimizer.zero_grad()
            y_pred = model(X_batch)
            loss = criterion(y_pred, y_batch)
            loss.backward()
            optimizer.step()
    
    # Evaluate
    model.eval()
    with torch.no_grad():
        y_pred_dl = model(X_train_dl.to(device)).cpu().numpy().flatten()
    
    # Calculate all metrics
    metrics = calculate_all_metrics(y_train_np, y_pred_dl)
    metrics['Waktu Pelatihan (s)'] = round(time.time() - start_time, 2)
    
    return metrics


In [12]:
 #12 TRAIN ALL MODELS

print("\n[STEP 3] Training all models...")

# Deep Learning Models
dl_models = {
    "LSTM": LSTMModel(EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM),
    "CNN": CNNModel(EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM),
    "LSTM-Attention": LSTMAttentionModel(EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM)
}

dl_results = []
for name, model in dl_models.items():
    print(f"\n🔥 Training: {name}")
    metrics = train_dl_model(model, X_train, y_train, NUM_EPOCHS)
    metrics['Model'] = name
    dl_results.append(metrics)
    print(f"  ✅ MSE: {metrics['MSE']} | R²: {metrics['R2']} | MAE: {metrics['MAE']} | RMSE: {metrics['RMSE']} | QWK: {metrics['QWK']}")

# Machine Learning Models
ml_models = {
    "Random Forest": RandomForestRegressor(random_state=42, n_estimators=100),
    "Support Vector Machines (SVR)": SVR(kernel='rbf'),
    "k-Nearest Neighbor (KNN)": KNeighborsRegressor(n_neighbors=5),
    "eXtreme Gradient Boosting (XGBoost)": XGBRegressor(random_state=42, n_estimators=100)
}

ml_results = []
for name, model in ml_models.items():
    print(f"\n🔥 Training: {name}")
    start_time = time.time()
    
    model.fit(X_train_flat, y_train_np)
    y_pred_ml = model.predict(X_train_flat)
    
    # Calculate all metrics
    metrics = calculate_all_metrics(y_train_np, y_pred_ml)
    metrics['Model'] = name
    metrics['Waktu Pelatihan (s)'] = round(time.time() - start_time, 2)
    
    ml_results.append(metrics)
    print(f"  ✅ MSE: {metrics['MSE']} | R²: {metrics['R2']} | MAE: {metrics['MAE']} | RMSE: {metrics['RMSE']} | QWK: {metrics['QWK']}")



[STEP 3] Training all models...

🔥 Training: LSTM


Training LSTMModel: 100%|██████████| 10/10 [00:03<00:00,  2.52it/s]


  ✅ MSE: 7.403500080108643 | R²: 0.9082 | MAE: 1.513800024986267 | RMSE: 2.720900058746338 | QWK: 0.951

🔥 Training: CNN


Training CNNModel: 100%|██████████| 10/10 [00:02<00:00,  3.67it/s]


  ✅ MSE: 12.593299865722656 | R²: 0.8439 | MAE: 1.9668999910354614 | RMSE: 3.5487000942230225 | QWK: 0.916

🔥 Training: LSTM-Attention


Training LSTMAttentionModel: 100%|██████████| 10/10 [00:04<00:00,  2.32it/s]


  ✅ MSE: 7.310299873352051 | R²: 0.9094 | MAE: 1.5348000526428223 | RMSE: 2.7037999629974365 | QWK: 0.9528

🔥 Training: Random Forest
  ✅ MSE: 3.2485 | R²: 0.9597 | MAE: 0.916 | RMSE: 1.8024 | QWK: 0.9773

🔥 Training: Support Vector Machines (SVR)
  ✅ MSE: 23.4018 | R²: 0.7099 | MAE: 2.0902 | RMSE: 4.8375 | QWK: 0.8125

🔥 Training: k-Nearest Neighbor (KNN)
  ✅ MSE: 11.851200103759766 | R²: 0.8531 | MAE: 1.5812000036239624 | RMSE: 3.4426000118255615 | QWK: 0.9184

🔥 Training: eXtreme Gradient Boosting (XGBoost)
  ✅ MSE: 0.38199999928474426 | R²: 0.9953 | MAE: 0.47929999232292175 | RMSE: 0.6179999709129333 | QWK: 0.9971


In [13]:
#13 FINAL COMPARISON TABLE

print("\n" + "="*80)
print("🏆 FINAL RESULTS COMPARISON")
print("="*80)

df_comparison = pd.DataFrame(dl_results + ml_results)

# Reorder columns to match your format
df_comparison = df_comparison[['Model', 'MSE', 'R2', 'MAE', 'RMSE', 'QWK', 'Waktu Pelatihan (s)']]

# Sort by QWK (highest is best)
df_comparison = df_comparison.sort_values(by="QWK", ascending=False)

print("\n" + df_comparison.to_markdown(index=False))
print("="*80)

# Best model
best_model_name = df_comparison.iloc[0]["Model"]
best_qwk = df_comparison.iloc[0]["QWK"]
best_r2 = df_comparison.iloc[0]["R2"]
best_rmse = df_comparison.iloc[0]["RMSE"]

print(f"\n🏆 BEST MODEL: {best_model_name}")
print(f"   - QWK: {best_qwk}")
print(f"   - R²: {best_r2}")
print(f"   - RMSE: {best_rmse}")
print("\n✅ TRAINING COMPLETED!")



🏆 FINAL RESULTS COMPARISON

| Model                               |     MSE |     R2 |    MAE |   RMSE |    QWK |   Waktu Pelatihan (s) |
|:------------------------------------|--------:|-------:|-------:|-------:|-------:|----------------------:|
| eXtreme Gradient Boosting (XGBoost) |  0.382  | 0.9953 | 0.4793 | 0.618  | 0.9971 |                  6.8  |
| Random Forest                       |  3.2485 | 0.9597 | 0.916  | 1.8024 | 0.9773 |                497.89 |
| LSTM-Attention                      |  7.3103 | 0.9094 | 1.5348 | 2.7038 | 0.9528 |                  4.39 |
| LSTM                                |  7.4035 | 0.9082 | 1.5138 | 2.7209 | 0.951  |                  4.05 |
| k-Nearest Neighbor (KNN)            | 11.8512 | 0.8531 | 1.5812 | 3.4426 | 0.9184 |                  0.77 |
| CNN                                 | 12.5933 | 0.8439 | 1.9669 | 3.5487 | 0.916  |                  2.75 |
| Support Vector Machines (SVR)       | 23.4018 | 0.7099 | 2.0902 | 4.8375 | 0.8125 |      

In [14]:
# 14 SAVE ALL MODELS

print("\n" + "="*80)
print("💾 SAVING MODELS")
print("="*80)

import os
os.makedirs('models', exist_ok=True)

# Save Deep Learning models (.pth)
print("\n📦 Saving Deep Learning models (.pth)...")
for name, model in dl_models.items():
    model_path = f"models/{name.replace(' ', '_')}.pth"
    
    # Save model state dict + architecture info
    torch.save({
        'model_name': name,
        'model_state_dict': model.state_dict(),
        'embedding_dim': EMBEDDING_DIM,
        'hidden_dim': HIDDEN_DIM,
        'output_dim': OUTPUT_DIM,
        'metrics': next((item for item in dl_results if item['Model'] == name), None)
    }, model_path)
    
    print(f"  ✅ Saved: {model_path}")

# Save Machine Learning models (.pkl)
print("\n📦 Saving Machine Learning models (.pkl)...")
import pickle

for name, model in ml_models.items():
    model_path = f"models/{name.replace(' ', '_')}.pkl"
    
    with open(model_path, 'wb') as f:
        pickle.dump({
            'model_name': name,
            'model': model,
            'metrics': next((item for item in ml_results if item['Model'] == name), None)
        }, f)
    
    print(f"  ✅ Saved: {model_path}")

# Save best model specifically
print("\n🏆 Saving best model...")
if best_model_name in dl_models:
    # Deep Learning model
    best_model_path = f"models/BEST_MODEL_{best_model_name.replace(' ', '_')}.pth"
    torch.save({
        'model_name': best_model_name,
        'model_state_dict': dl_models[best_model_name].state_dict(),
        'embedding_dim': EMBEDDING_DIM,
        'hidden_dim': HIDDEN_DIM,
        'output_dim': OUTPUT_DIM,
        'metrics': df_comparison.iloc[0].to_dict(),
        'model_type': 'deep_learning'
    }, best_model_path)
else:
    # Machine Learning model
    best_model_path = f"models/BEST_MODEL_{best_model_name.replace(' ', '_')}.pkl"
    with open(best_model_path, 'wb') as f:
        pickle.dump({
            'model_name': best_model_name,
            'model': ml_models[best_model_name],
            'metrics': df_comparison.iloc[0].to_dict(),
            'model_type': 'machine_learning'
        }, f)

print(f"  ✅ Saved best model: {best_model_path}")

# Save comparison results
df_comparison.to_csv('models/comparison_results.csv', index=False)
print(f"  ✅ Saved comparison table: models/comparison_results.csv")

# Save metadata
metadata = {
    'best_model': best_model_name,
    'best_qwk': best_qwk,
    'best_r2': best_r2,
    'best_rmse': best_rmse,
    'embedding_dim': EMBEDDING_DIM,
    'hidden_dim': HIDDEN_DIM,
    'num_epochs': NUM_EPOCHS,
    'batch_size': DL_BATCH_SIZE,
    'sbert_model': 'paraphrase-MiniLM-L6-v2'
}

with open('models/metadata.pkl', 'wb') as f:
    pickle.dump(metadata, f)

print(f"  ✅ Saved metadata: models/metadata.pkl")

print("\n" + "="*80)
print("✅ ALL MODELS SAVED SUCCESSFULLY!")
print("="*80)
print("\n📁 Saved files:")
print("  - models/*.pth (Deep Learning models)")
print("  - models/*.pkl (Machine Learning models)")
print("  - models/BEST_MODEL_*.pth or .pkl")
print("  - models/comparison_results.csv")
print("  - models/metadata.pkl")

print("\n✅ TRAINING COMPLETED!")



💾 SAVING MODELS

📦 Saving Deep Learning models (.pth)...
  ✅ Saved: models/LSTM.pth
  ✅ Saved: models/CNN.pth
  ✅ Saved: models/LSTM-Attention.pth

📦 Saving Machine Learning models (.pkl)...
  ✅ Saved: models/Random_Forest.pkl
  ✅ Saved: models/Support_Vector_Machines_(SVR).pkl
  ✅ Saved: models/k-Nearest_Neighbor_(KNN).pkl
  ✅ Saved: models/eXtreme_Gradient_Boosting_(XGBoost).pkl

🏆 Saving best model...
  ✅ Saved best model: models/BEST_MODEL_eXtreme_Gradient_Boosting_(XGBoost).pkl
  ✅ Saved comparison table: models/comparison_results.csv
  ✅ Saved metadata: models/metadata.pkl

✅ ALL MODELS SAVED SUCCESSFULLY!

📁 Saved files:
  - models/*.pth (Deep Learning models)
  - models/*.pkl (Machine Learning models)
  - models/BEST_MODEL_*.pth or .pkl
  - models/comparison_results.csv
  - models/metadata.pkl

✅ TRAINING COMPLETED!


In [ ]:
# =======================================================
# 🎯 HYBRID AI GRADING SYSTEM (OPENROUTER FIXED - CLEAN)
# =======================================================

import gradio as gr
import joblib
import pandas as pd # Tambahan impor
import numpy as np # Tambahan impor
from sklearn.metrics.pairwise import cosine_similarity
import openai 
import json
import os # Tambahan impor
import torch # Tambahan impor
import torch.nn as nn # Tambahan impor
from sentence_transformers import SentenceTransformer # Tambahan impor
from xgboost import XGBRegressor # Tambahan impor

print("="*60)
print("🚀 INITIALIZING HYBRID AI GRADING SYSTEM (OpenRouter Mode)")
print("="*60)

# =====================================================
# 🔧 1. LOAD SBERT MODEL
# =====================================================

print("\n🔄 Loading SBERT model...")
try:
    # Muat model SBERT
    encoder = SentenceTransformer("paraphrase-MiniLM-L6-v2")
    print("✅ SBERT model loaded successfully!")
except Exception as e:
    print(f"❌ Error loading SBERT: {e}")
    encoder = None

# =====================================================
# 🔧 2. PLACEHOLDER DL MODEL CLASS
# =====================================================

class MyModel(torch.nn.Module):
    def __init__(self, input_dim=384, hidden_dim=128, output_dim=1):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        return self.fc2(x)

# =====================================================
# 🔧 3. LOAD TRAINED MODEL (ML/DL)
# =====================================================

def get_dummy_model():
    """Fallback jika model tidak ditemukan"""
    print("⚠️ Model asli tidak ditemukan, menggunakan DUMMY Model")
    dummy = XGBRegressor(random_state=42)
    # Fit dengan data random agar bisa .predict()
    dummy.fit(np.random.rand(10, 384), np.random.rand(10) * 12)
    return dummy, "ml", 12.0 # Max score dataset ASAP-AES

def load_custom_model():
    model_dir = "models"
    
    if not os.path.exists(model_dir):
        os.makedirs(model_dir, exist_ok=True)
        return get_dummy_model()
    
    try:
        files = os.listdir(model_dir)
        
        best_pkl = next((os.path.join(model_dir, f) for f in files if "BEST_MODEL" in f and f.endswith(".pkl")), None)
        best_pth = next((os.path.join(model_dir, f) for f in files if "BEST_MODEL" in f and f.endswith(".pth")), None)
        
        if best_pkl:
            print(f"🚀 Loading ML Model: {best_pkl}")
            data = joblib.load(best_pkl)
            model = data.get('model', data.get('estimator', data)) if isinstance(data, dict) else data
            if model is None:
                 print("❌ Model tidak ditemukan dalam dict")
                 return get_dummy_model()
            print("✅ ML Model loaded successfully!")
            return model, "ml", 12.0
        
        if best_pth:
            print(f"🚀 Loading DL Model: {best_pth}")
            model = MyModel()
            checkpoint = torch.load(best_pth, map_location="cpu")
            state_dict = checkpoint.get('model_state_dict', checkpoint) if isinstance(checkpoint, dict) else checkpoint
            model.load_state_dict(state_dict)
            model.eval()
            print("✅ DL Model loaded successfully!")
            return model, "dl", 12.0
            
        return get_dummy_model()
        
    except Exception as e:
        print(f"❌ Error loading model: {e}")
        return get_dummy_model()

# Load model global
best_model, model_type, model_max_score = load_custom_model()
print(f"\n📊 Model Type: {model_type.upper()}")
print(f"📊 Max Score Range: 0-{model_max_score}")

# =====================================================
# 🧠 4. OPENROUTER LLM CHECKER (via OpenAI standard)
# =====================================================

def cek_konteks_llm(api_key, soal, kunci, jawaban_mhs):
    if not api_key or not api_key.strip() or not jawaban_mhs.strip():
        return 0, "⚠️ API Key/Jawaban kosong. Mode Offline."
    
    try:
        # Inisialisasi client OpenRouter (menggunakan format OpenAI)
        client = openai.OpenAI(
            base_url="https://openrouter.ai/api/v1",
            api_key=api_key,
        )

        OPENROUTER_MODEL = "mistralai/mistral-7b-instruct:free" 
        
        prompt = f"""
Kamu adalah Dosen Penilai yang Objektif dan Teliti.

**SOAL:**
{soal}

**KUNCI JAWABAN (Reference):**
{kunci}

**JAWABAN MAHASISWA:**
{jawaban_mhs}

**TUGAS:**
Berikan penilaian objektif berdasarkan kriteria: Relevansi (40%), Kebenaran (40%), Kelengkapan (20%).
Hanya beri skor rendah jika benar-benar salah/tidak relevan.

**OUTPUT (WAJIB FORMAT JSON):**
{{"skor": <integer 0-100>, "alasan": "<penjelasan singkat 1-2 kalimat>"}}
"""
        
        response = client.chat.completions.create(
            model=OPENROUTER_MODEL,
            messages=[
                {"role": "system", "content": "You are an objective essay grader. Your output must be a single JSON object. Only output the JSON."},
                {"role": "user", "content": prompt}
            ],
            response_format={"type": "json_object"}, 
            temperature=0.3
        )
        
        text_res = response.choices[0].message.content.strip()
        data = json.loads(text_res)
        skor = max(0, min(100, int(data.get('skor', 0))))
        alasan = data.get('alasan', 'Tidak ada feedback')
        return skor, alasan
        
    except json.JSONDecodeError:
        import re
        match = re.search(r'"skor"\s*:\s*(\d+)', text_res)
        if match:
            skor = max(0, min(100, int(match.group(1))))
            return skor, "Feedback parsing error, skor diekstrak manual"
        else:
            return 0, f"⚠️ Gagal parse response OpenRouter: {text_res[:100]}..."
            
    except openai.APIError as e:
        error_msg = str(e)
        if "rate limit" in error_msg.lower() or "429" in error_msg:
             return 0, "❌ OpenRouter Error: Quota/Rate Limit habis. Sistem fallback ke mode Offline."
        elif "authentication" in error_msg.lower() or "401" in error_msg:
             return 0, "❌ OpenRouter Error: API Key tidak valid."
        print(f"❌ OpenRouter API Error: {error_msg}")
        return 0, f"❌ OpenRouter Error: {error_msg[:50]}... | Mode fallback aktif."

    except Exception as e:
        error_msg = str(e)
        print(f"❌ Unexpected OpenRouter Error: {error_msg}")
        return 0, f"❌ Error tak terduga: {error_msg[:50]}... | Mode fallback aktif."


# =====================================================
# 🎯 5. HYBRID GRADING FUNCTION (CORE LOGIC)
# =====================================================

def grade_essay_hybrid(api_key, jawaban_mahasiswa, current_state):
    if not jawaban_mahasiswa or not jawaban_mahasiswa.strip():
        return 0.0, 0.0, 0.0, "Jawaban kosong", "N/A"
        
    soal = current_state.get("soal", "")
    kunci_jawaban = current_state.get("jawaban_benar", "")
    max_score_dosen = current_state.get("max_score", 100)
    
    # STEP 1: TECHNICAL SCORING (Model-based)
    try:
        emb_mhs = encoder.encode([jawaban_mahasiswa])
        emb_kunci = encoder.encode([kunci_jawaban])
        
        similarity = cosine_similarity(emb_mhs, emb_kunci)[0][0]
        similarity = max(0.0, similarity)
        
        if model_type == "ml":
            raw_pred = best_model.predict(emb_mhs)[0]
        else:
            t_mhs = torch.tensor(emb_mhs, dtype=torch.float32)
            with torch.no_grad():
                raw_pred = best_model(t_mhs).item()
        
        score_technical = (float(raw_pred) / model_max_score) * 100
        score_technical = max(0.0, min(100.0, score_technical))
        if similarity > 0.99:
            score_technical = 100.0
    
    except Exception as e:
        print(f"❌ Technical scoring error: {e}")
        score_technical = 0.0
        similarity = 0.0
    
    # STEP 2: LOGICAL SCORING (LLM-based)
    score_llm, feedback_llm = cek_konteks_llm(api_key, soal, kunci_jawaban, jawaban_mahasiswa)
    
    # STEP 3: FINAL SCORE CALCULATION
    llm_success = (score_llm > 0 or "Error" not in feedback_llm) and api_key and "Offline" not in feedback_llm
    
    if llm_success:
        # HYBRID MODE (Online with LLM): 40% Technical + 60% LLM
        final_score_raw = (score_technical * 0.4) + (score_llm * 0.6)
        mode_used = "🟢 Hybrid (AI+Model)"
    else:
        # OFFLINE MODE (Fallback tanpa LLM): 50% Model + 50% Similarity
        similarity_score = similarity * 100
        final_score_raw = (score_technical * 0.5) + (similarity_score * 0.5)
        mode_used = "🟡 Offline (Model Only)"
    
    # Scale ke max_score dosen
    final_score = (final_score_raw / 100.0) * max_score_dosen
    final_score = max(0.0, min(max_score_dosen, final_score))
    
    return (
        round(final_score, 2),
        round(score_technical, 2),
        round(score_llm, 2),
        feedback_llm,
        mode_used
    )

# =====================================================
# 🗂️ 6. GRADIO UI LOGIC FUNCTIONS
# =====================================================

def simpan_soal(soal, jawaban_benar, max_score, state):
    """Handler untuk tombol Simpan Soal"""
    if not soal or not soal.strip() or not jawaban_benar or not jawaban_benar.strip():
        empty_df = pd.DataFrame(columns=["Nama", "Total", "Teknis", "Logika(AI)", "Mode", "Feedback"])
        return state, "❌ Soal/Kunci Jawaban tidak boleh kosong!", empty_df
    
    new_state = {
        "soal": soal,
        "jawaban_benar": jawaban_benar,
        "max_score": float(max_score) if max_score else 100.0,
        "leaderboard": pd.DataFrame(columns=["Nama", "Total", "Teknis", "Logika(AI)", "Mode", "Feedback"])
    }
    
    status_msg = f"✅ Soal tersimpan!\n📝 Soal: {soal[:50]}...\n🎯 Max Score: {new_state['max_score']}"
    
    return new_state, status_msg, new_state["leaderboard"]


def submit_jawaban_hybrid(api_key, nama, jawaban, state):
    """Handler untuk tombol Submit Jawaban"""
    if not state or "soal" not in state or not state["soal"]:
        return state, state.get("leaderboard", pd.DataFrame()), "⚠️ **Error:** Dosen belum membuat soal!"
    
    if not nama or not nama.strip():
        return state, state.get("leaderboard", pd.DataFrame()), "⚠️ **Error:** Nama harus diisi!"
    
    if not jawaban or not jawaban.strip():
        return state, state.get("leaderboard", pd.DataFrame()), "⚠️ **Error:** Jawaban harus diisi!"
    
    total, teknis, logika, feedback, mode = grade_essay_hybrid(api_key, jawaban, state)
    
    # Update leaderboard
    df = state.get("leaderboard", pd.DataFrame(columns=["Nama", "Total", "Teknis", "Logika(AI)", "Mode", "Feedback"]))
    
    new_row = {
        "Nama": nama, "Total": total, "Teknis": teknis, 
        "Logika(AI)": logika, "Mode": mode, "Feedback": feedback
    }
    
    # Cek dan update/tambah row
    if not df.empty and nama in df["Nama"].values:
        df.loc[df["Nama"] == nama, list(new_row.keys())] = list(new_row.values())
    else:
        new_df = pd.DataFrame([new_row])
        df = pd.concat([df, new_df], ignore_index=True)
    
    df = df.sort_values(by="Total", ascending=False).reset_index(drop=True)
    state["leaderboard"] = df
    
    # Format feedback message
    max_score = state.get("max_score", 100)
    emoji = "📚"
    grade = "Perlu Belajar"
    if total >= max_score * 0.9:
        emoji = "🌟"
        grade = "Sempurna!"
    elif total >= max_score * 0.7:
        emoji = "✅"
        grade = "Bagus!"
    elif total >= max_score * 0.5:
        emoji = "👍"
        grade = "Cukup"
    
    msg = f"""
### {emoji} Hasil Penilaian: **{nama}**
---
**🏆 SKOR AKHIR: {total} / {max_score}** ({grade})
📊 **Rincian Penilaian:**
- 🤖 **Teknis (Model):** {teknis}/100 
- 🧠 **Logika (AI):** {logika}/100 
---
💡 **Feedback AI:** > {feedback}
🔧 **Mode:** {mode}
---
"""
    return state, df.drop(columns=["Feedback"]), msg # Leaderboard di mhs tidak tampilkan kolom Feedback


def tampilkan_soal(state):
    """Tampilkan soal di tab mahasiswa"""
    if state and "soal" in state and state["soal"]:
        return f"📋 **Soal:**\n\n{state['soal']}"
    return "⚠️ _(Dosen belum membuat soal)_"

# =====================================================
# 🎨 7. GRADIO UI (COMPLETE & CLEAN)
# =====================================================

with gr.Blocks(title="Hybrid AI Grader") as demo:
    initial_state = {
        "soal": None, "jawaban_benar": None, "max_score": 100.0,
        "leaderboard": pd.DataFrame(columns=["Nama", "Total", "Teknis", "Logika(AI)", "Mode", "Feedback"])
    }
    state = gr.State(initial_state)
    
    gr.Markdown("""
    # 🚀 Hybrid AI Essay Grading System
    ### Sistem Penilaian Esai dengan 3 Teknologi: 🤖 SBERT + 📊 ML Model + 🧠 LLM (OpenRouter)
    ---
    """)
    
    # API Key Setup
    with gr.Accordion("🔑 Setup API Key OpenRouter (Required for Smart Mode)", open=False):
        gr.Markdown("""
        **Cara Mendapatkan API Key OpenRouter:** [OpenRouter Dashboard](https://openrouter.ai/keys)
        ⚠️ **Tanpa API Key:** Sistem akan berjalan dalam mode Offline (akurasi berkurang)
        """)
        api_key_input = gr.Textbox(
            label="OpenRouter API Key", type="password", placeholder="sk-or-v1-...", 
            info="Key ini menggantikan Gemini Key. TIDAK akan disimpan."
        )
    
    with gr.Tabs():
        # TAB 1: PORTAL DOSEN
        with gr.Tab("👨‍🏫 Portal Dosen"):
            gr.Markdown("### Buat Soal & Kunci Jawaban")
            
            with gr.Row():
                with gr.Column(scale=2):
                    soal_in = gr.Textbox(label="📝 Soal Essay", lines=3, placeholder="Jelaskan dampak AI pada pendidikan modern...")
                    kunci_in = gr.Textbox(label="✅ Kunci Jawaban (Referensi Ideal)", lines=8, placeholder="Tuliskan jawaban ideal...")
                
                with gr.Column(scale=1):
                    max_score = gr.Number(value=100, label="🎯 Skor Maksimal")
                    btn_save = gr.Button("💾 Simpan Soal", variant="primary", size="lg")
                    status_dosen = gr.Textbox(label="📊 Status", interactive=False, lines=3)
            
            gr.Markdown("---")
            gr.Markdown("### 📊 Preview Leaderboard (Dosen)")
            leaderboard_dosen = gr.DataFrame(label="Leaderboard (Live)", interactive=False)
            
            # Connect button
            btn_save.click(
                fn=simpan_soal,
                inputs=[soal_in, kunci_in, max_score, state],
                outputs=[state, status_dosen, leaderboard_dosen]
            )
        
        # TAB 2: PORTAL MAHASISWA
        with gr.Tab("🧑‍🎓 Portal Mahasiswa") as mahasiswa_tab:
            gr.Markdown("### Lihat Soal & Submit Jawaban")
            
            soal_display = gr.Markdown(value="⚠️ _(Dosen belum membuat soal)_")
            
            with gr.Row():
                with gr.Column(scale=2):
                    nama_in = gr.Textbox(label="👤 Nama Lengkap", placeholder="Contoh: Budi Santoso")
                    jawab_in = gr.Textbox(label="✍️ Jawaban Kamu", lines=10, placeholder="Tuliskan jawabanmu di sini...")
                    btn_submit = gr.Button("🚀 Kirim Jawaban", variant="primary", size="lg")
                
                with gr.Column(scale=1):
                    gr.Markdown("### 🏆 Live Leaderboard (Top 5)")
                    lb_mhs = gr.DataFrame(label="Top Scores", interactive=False, headers=["Nama", "Total", "Teknis", "Logika(AI)"])
            
            gr.Markdown("---")
            status_mhs = gr.Markdown(value="_Isi form di atas dan klik 'Kirim Jawaban'_")
            
            # Connect button
            btn_submit.click(
                fn=submit_jawaban_hybrid,
                inputs=[api_key_input, nama_in, jawab_in, state],
                outputs=[state, lb_mhs, status_mhs]
            )
            
            # Update soal saat tab dibuka
            mahasiswa_tab.select(
                fn=tampilkan_soal,
                inputs=[state],
                outputs=[soal_display]
            )

# =====================================================
# 🚀 8. LAUNCH APPLICATION
# =====================================================

print("\n" + "="*60)
print("✅ SYSTEM READY!")
print("="*60)
print(f"📊 Model Type: {model_type.upper()}")
print(f"🤖 SBERT: {'✅ Loaded' if encoder else '❌ Failed'}")
print(f"🧠 LLM: OpenRouter (via Mistral 7B)")
print("="*60)
print("\n🚀 Launching Gradio interface...\n")

demo.launch(share=True, debug=True)

🚀 INITIALIZING HYBRID AI GRADING SYSTEM (OpenRouter Mode)

🔄 Loading SBERT model...
✅ SBERT model loaded successfully!
🚀 Loading ML Model: models/BEST_MODEL_eXtreme_Gradient_Boosting_(XGBoost).pkl
✅ ML Model loaded successfully!

📊 Model Type: ML
📊 Max Score Range: 0-12.0

✅ SYSTEM READY!
📊 Model Type: ML
🤖 SBERT: ✅ Loaded
🧠 LLM: OpenRouter (via Mistral 7B)

🚀 Launching Gradio interface...

* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://15a35e07a5a0d248f0.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
